In [2]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [3]:
import correlation_tool

# Axioms to test

Given a 7 pip movement of the 'base' symbols:

- If USDJPY, USDCHF rise, EURUSD falls 
- If GBPUSD and GBPJPY rise, EURGBP falls
- If AUDSD falls and USDCAD rises, Gold increases 
- If EURUSD rises and USDCHF falls, EURCHF increases 

## Notes

- Could there be a time-lagged correlation with the above? For example, instead of the 3rd symbol increasing/decreasing based on the first two symbols on the next candle, could the ith element increase instead? So if USDJPY and USDCHF rise, would EURUSD fall on the 6th candle following?

In [137]:
#USDJPY, USDCHF VS EURUSD
UJ = correlation_tool.get_rates('USDJPY.a', mt5.TIMEFRAME_M15, 5000)
UJ['UJ_Return'] = (UJ['open'] - UJ['close']) / 100
UC = correlation_tool.get_rates('USDCHF.a', mt5.TIMEFRAME_M15, 5000)
UC['UC_Return'] = (UC['open'] - UC['close']) 
df = pd.DataFrame(UJ['UJ_Return'])
df2 = pd.DataFrame(UC['UC_Return'])
comb_df = pd.concat([df, df2], axis = 1)
pip_threshold = 0.0005
filtered_df = comb_df[((comb_df['UJ_Return'] > pip_threshold) | (comb_df['UJ_Return'] < -pip_threshold))
                      & ((comb_df['UC_Return'] > pip_threshold) | (comb_df['UC_Return'] < -pip_threshold))]
# Print the filtered dataframe
filtered_df.head(10)

,UJ_Return,UC_Return
time,,
2023-05-01 10:00:00,0.00096,0.00110
2023-05-01 15:30:00,0.00095,0.00102
2023-05-01 17:30:00,-0.00074,-0.00148
2023-05-01 18:00:00,-0.00095,-0.00092
2023-05-02 00:00:00,0.00054,-0.00076
2023-05-02 04:00:00,0.00063,0.00074
2023-05-02 08:45:00,0.00063,-0.00056
2023-05-02 11:15:00,0.00077,0.00053
2023-05-02 11:45:00,-0.00095,-0.00069


In [139]:
EU = correlation_tool.get_rates('EURUSD.a', mt5.TIMEFRAME_M15, 5000)
EU['EU_Return'] = (EU['open'] - EU['close'])
EU['Shifted'] = EU['EU_Return'].shift(periods = 1)
EU

,open,high,low,close,EU_Return,Shifted
time,,,,,,
2023-05-01 07:30:00,1.10136,1.10137,1.10084,1.10085,0.00051,NaN
2023-05-01 07:45:00,1.10085,1.10089,1.10033,1.10064,0.00021,0.00051
2023-05-01 08:00:00,1.10062,1.10062,1.10038,1.10046,0.00016,0.00021
2023-05-01 08:15:00,1.10046,1.10061,1.10027,1.10034,0.00012,0.00016
2023-05-01 08:30:00,1.10029,1.10079,1.10009,1.10074,-0.00045,0.00012
...,...,...,...,...,...,...
2023-07-12 08:15:00,1.10332,1.10353,1.10303,1.10321,0.00011,0.00030
2023-07-12 08:30:00,1.10321,1.10325,1.10267,1.10299,0.00022,0.00011
2023-07-12 08:45:00,1.10300,1.10308,1.10255,1.10260,0.00040,0.00022


In [144]:
new_df = pd.DataFrame()
for i in EU.index:
    if i in filtered_df.index:
        eu_row = EU.loc[[i]]
        eu = eu_row['Shifted']
        new_df = EU.loc[filtered_df.index, 'Shifted']
        new_df = new_df.to_frame()  # Convert the Series to a DataFrame
# Append the row to the 'filtered_df'
final_df = pd.concat([filtered_df, new_df], axis = 1)

#Print the updated 'filtered_df'
final_df

# This df represents the return of EURUSD whenever USDJPY and USDCHF both rise by a value of more than 6 pips.

,UJ_Return,UC_Return,Shifted
time,,,
2023-05-01 10:00:00,0.00096,0.00110,0.00040
2023-05-01 15:30:00,0.00095,0.00102,-0.00048
2023-05-01 17:30:00,-0.00074,-0.00148,0.00105
2023-05-01 18:00:00,-0.00095,-0.00092,-0.00010
2023-05-02 00:00:00,0.00054,-0.00076,-0.00005
...,...,...,...
2023-07-10 17:00:00,0.00100,0.00195,-0.00092
2023-07-10 17:30:00,-0.00112,-0.00063,0.00050
2023-07-11 09:15:00,0.00123,0.00119,-0.00082


In [146]:
t1 = final_df
# t1['EU_Return_Lagged'] = t1['EU_Return'].shift(periods = 1)
t1['UJ+UC'] = t1['UJ_Return'] + t1['UC_Return']
t1.corr()

,UJ_Return,UC_Return,Shifted,UJ+UC
UJ_Return,1.000000,0.743770,-0.046408,0.961496
UC_Return,0.743770,1.000000,-0.043527,0.898831
Shifted,-0.046408,-0.043527,1.000000,-0.048325
UJ+UC,0.961496,0.898831,-0.048325,1.000000


In [ ]:
#Not a true EU_Return Lag. Need to go back to the EU dataframe and shift the values there.